In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

home_rate = .14
public_rate = 0.30
public_activation_fee = 1.50

gas = 3.40
mpg = 24.4

In [30]:
charging_path = "charging.csv"
driving_path = "driving.csv"

charging_data = pd.read_csv(charging_path)
driving_data = pd.read_csv(driving_path)



driving_data.head()


,date,start_time,start_bat,end_time,end_bat,miles,average_m/kwh
0,3/31/2025,8:34,95,9:26,82,21.4,5.5
1,3/31/2025,16:46,82,17:23,70,14.2,4.7
2,3/31/2025,20:16,70,20:53,46,30.3,4.5
3,4/1/2025,8:25,95,9:24,81,23.5,5.3
4,4/1/2025,16:42,81,16:03,60,30.0,5.0


In [31]:
charging_data.head()

,date,charge_place,charge_time,charge_amount
0,3/31/2025,Home,1:58:35,13.4
1,4/1/2025,Home,2:28:12,16.8
2,4/2/2025,Home,1:28:56,10.0
3,4/3/2025,Home,1:58:33,13.4
4,4/4/2025,Home,1:58:31,13.4


In [32]:
def calculate_session_cost(row):
    if row["charge_place"].lower() == "home":
        return row["charge_amount"] * home_rate
    elif row["charge_place"].lower() == "public":
        return (row["charge_amount"] * public_rate) + public_activation_fee
    else:
        return 0

In [33]:
charging_data["cost"] = charging_data.apply(calculate_session_cost, axis=1)

In [34]:
charging_data["cost"] = charging_data["cost"].round(2)

In [35]:
print(charging_data)

        date charge_place charge_time  charge_amount  cost
0  3/31/2025         Home     1:58:35           13.4  1.88
1   4/1/2025         Home     2:28:12           16.8  2.35
2   4/2/2025         Home     1:28:56           10.0  1.40
3   4/3/2025         Home     1:58:33           13.4  1.88
4   4/4/2025         Home     1:58:31           13.4  1.88


In [38]:
driving_data["trip_cost"] = (driving_data["miles"] / mpg) * gas
driving_data["trip_cost"] = driving_data["trip_cost"].round(2)

In [39]:
driving_data["date"] = pd.to_datetime(driving_data["date"])

daily_summary = driving_data.groupby("date", as_index=False).agg({
    "miles": "sum",
    "trip_cost": "sum"
})

# Round trip_cost for neatness
daily_summary["trip_cost"] = daily_summary["trip_cost"].round(2)

# Show result
print(daily_summary)

        date  miles  trip_cost
0 2025-03-31   65.9       9.18
1 2025-04-01   53.5       7.45
2 2025-04-02   50.6       7.05
3 2025-04-03   49.6       6.91
4 2025-04-04   49.7       6.92


In [42]:
average_cost = driving_data["trip_cost"].mean()
average_charge = charging_data["cost"].mean()

total_cost = driving_data["trip_cost"].sum()
total_charge = charging_data["cost"].sum()

In [43]:
print(f"The average cost if I had a gas car would be: ${average_cost:.2f} and the average charge is ${average_charge:.2f}")
print(f"The total cost if I had a gas car would be: ${total_cost:.2f} and the total charge is ${total_charge:.2f}")

The average cost if I had a gas car would be: $3.41 and the average charge is $1.88
The total cost if I had a gas car would be: $37.51 and the total charge is $9.39
